# Example of using the gnomAD genetic ancestry principal components analysis loadings and random forest classifier

Please read our [blog post](https://gnomad.broadinstitute.org/news/2021-09-using-the-gnomad-ancestry-principal-components-analysis-loadings-and-random-forest-classifier-on-your-dataset/) about important caveats to consider when using gnomAD ancestry principal components analysis (PCA) loadings and random forest classifier models on your own dataset.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#The-following-packages-are-required-for-this-example" data-toc-modified-id="The-following-packages-are-required-for-this-example-2">The following packages are required for this example</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-3">Imports</a></span></li><li><span><a href="#Data-Loading" data-toc-modified-id="Data-Loading-4">Data Loading</a></span><ul class="toc-item"><li><span><a href="#Define-file-paths" data-toc-modified-id="Define-file-paths-4.1">Define file paths</a></span><ul class="toc-item"><li><span><a href="#v3-example-paths" data-toc-modified-id="v3-example-paths-4.1.1">v3 example paths</a></span></li><li><span><a href="#v2-example-paths" data-toc-modified-id="v2-example-paths-4.1.2">v2 example paths</a></span></li></ul></li><li><span><a href="#Define-the-number-of-PCs-used-for-v2-and-v3-genetic-ancestry-group-classification" data-toc-modified-id="Define-the-number-of-PCs-used-for-v2-and-v3-genetic-ancestry-group-classification-4.2">Define the number of PCs used for v2 and v3 genetic ancestry group classification</a></span></li><li><span><a href="#Define-the-RF-minimum-probability-used-for-v2-and-v3-genetic-ancestry-group-classification" data-toc-modified-id="Define-the-RF-minimum-probability-used-for-v2-and-v3-genetic-ancestry-group-classification-4.3">Define the RF minimum probability used for v2 and v3 genetic ancestry group classification</a></span></li><li><span><a href="#Load-ONNX-models" data-toc-modified-id="Load-ONNX-models-4.4">Load ONNX models</a></span></li><li><span><a href="#Load-gnomAD-v3.1-loadings-Hail-Table-and-the-VariantDataset-to-apply-projection-and-genetic-ancestry-group-assignment-to" data-toc-modified-id="Load-gnomAD-v3.1-loadings-Hail-Table-and-the-VariantDataset-to-apply-projection-and-genetic-ancestry-group-assignment-to-4.5">Load gnomAD v3.1 loadings Hail Table and the VariantDataset to apply projection and genetic ancestry group assignment to</a></span></li><li><span><a href="#Load-gnomAD-v2.1-precomputed-v2-PCA-scores" data-toc-modified-id="Load-gnomAD-v2.1-precomputed-v2-PCA-scores-4.6">Load gnomAD v2.1 precomputed v2 PCA scores</a></span></li></ul></li><li><span><a href="#Perform-PC-projection-using-the-v3.1-PCA-loadings" data-toc-modified-id="Perform-PC-projection-using-the-v3.1-PCA-loadings-5">Perform PC projection using the v3.1 PCA loadings</a></span><ul class="toc-item"><li><span><a href="#Create-dense-MatrixTable-of-only-the-variants-found-in-the-loadings-Table" data-toc-modified-id="Create-dense-MatrixTable-of-only-the-variants-found-in-the-loadings-Table-5.1">Create dense MatrixTable of only the variants found in the loadings Table</a></span></li><li><span><a href="#We-recommend-filtering-to-entries-meeting-GQ,-DP-and-het-AB-'adj'-thresholds" data-toc-modified-id="We-recommend-filtering-to-entries-meeting-GQ,-DP-and-het-AB-'adj'-thresholds-5.2">We recommend filtering to entries meeting GQ, DP and het AB 'adj' thresholds</a></span></li><li><span><a href="#Checkpoint-dense-MT-for-PC-projection" data-toc-modified-id="Checkpoint-dense-MT-for-PC-projection-5.3">Checkpoint dense MT for PC projection</a></span></li><li><span><a href="#Project-test-dataset-genotypes-onto-gnomAD-v3.1-loadings-and-checkpoint-the-scores" data-toc-modified-id="Project-test-dataset-genotypes-onto-gnomAD-v3.1-loadings-and-checkpoint-the-scores-5.4">Project test dataset genotypes onto gnomAD v3.1 loadings and checkpoint the scores</a></span></li></ul></li><li><span><a href="#Assign-genetic-ancestry-group-using-ONNX-RF-model" data-toc-modified-id="Assign-genetic-ancestry-group-using-ONNX-RF-model-6">Assign genetic ancestry group using ONNX RF model</a></span><ul class="toc-item"><li><span><a href="#v3.1-RF-model" data-toc-modified-id="v3.1-RF-model-6.1">v3.1 RF model</a></span></li><li><span><a href="#v2.1-RF-model" data-toc-modified-id="v2.1-RF-model-6.2">v2.1 RF model</a></span></li></ul></li></ul></div>

## Introduction

In this notebook, we show examples for how to use the gnomAD genetic ancestry PCA loadings and random forest (RF) classifiers.
- For the example using the [v3 release files](https://gnomad.broadinstitute.org/downloads#v3-ancestry-classification), we assign the genetic ancestry for a test dataset of gnomAD v4 using the v3.1 PCA loadings and the v3.1 ONNX formatted RF model. 
  - We start with the test dataset in raw VariantDataset (VDS) format and reduce it to only variants with v3.1 loading info using the following steps:
      - Split multiallelic variants.
      - Filter to variants found in the v3.1 loadings HailTable (HT).
      - Densify the VDS to a MatrixTable (MT).
      - Filter the MT to high quality genotypes (GQ >= 20; DP >= 10; and AB >= 0.2 for het calls).
  - Then, project the genotypes onto v3.1 PCs using the v3.1 loadings HT.
  - Assign the genetic ancestry using the v3.1 ONNX formatted RF model.
  - **NOTE: the reference genome for the v3.1 loadings HT is GRCh38.**
- For the example using the [v2 release files](https://gnomad.broadinstitute.org/downloads#v2-ancestry-classification), we use our precomputed v2.1 PCA scores to assign the genetic ancestry of the v2 samples with the v2.1 ONNX formatted RF model. 
   - For use on your own data, you can use the same process as shown for v3.1 to obtain the projected PCs. 
   - **NOTE: the reference genome for the v2.1 loadings HT is GRCh37.**

## The following packages are required for this example

In [1]:
!/opt/conda/default/bin/pip install onnxruntime onnx

## Imports

In [2]:
import onnx
import hail as hl
from gnomad.sample_qc.ancestry import apply_onnx_classification_model, assign_population_pcs
from gnomad.utils.filtering import filter_to_adj

from gnomad_qc.v2.resources.basics import get_gnomad_meta
from gnomad_qc.v4.resources.basics import get_checkpoint_path



## Data Loading

In [3]:
read_if_exists = True

### Define file paths

#### v3 example paths

In [4]:
# v3.1 PCA loadings.
gnomad_v3_loadings = "gs://gcp-public-data--gnomad/release/3.1/pca/gnomad.v3.1.pca_loadings.ht"

# v3.1 ONNX RF model.
gnomad_v3_onnx_rf = "gs://gcp-public-data--gnomad/release/3.1/pca/gnomad.v3.1.RF_fit.onnx"

# Test dataset to apply projection and genetic ancestry group assignment to.
# This will be the path to your dataset VDS.
vds_to_project = "gs://gnomad/v4.0/raw/exomes/testing/gnomad_v4.0_test.vds"

# v3.1 output paths.
test_mt_output_path = get_checkpoint_path("example_gnomad_v3.1_ancestry_rf", mt=True)
test_scores_output_path = get_checkpoint_path("example_gnomad_v3.1_ancestry_rf.scores")
gnomad_v3_assignment_path = get_checkpoint_path("example_gnomad_v3.1_ancestry_rf.assignment")

#### v2 example paths

In [5]:
# v2.1 ONNX RF model.
gnomad_v2_onnx_rf = "gs://gcp-public-data--gnomad/release/2.1/pca/gnomad.r2.1.RF_fit.onnx"

# v2.1 output path.
gnomad_v2_assignment_path = get_checkpoint_path("example_gnomad_v2.1_ancestry_rf.assignment")

### Define the number of PCs used for v2 and v3 genetic ancestry group classification

In [6]:
v3_num_pcs = 16
v2_num_pcs = 6

### Define the RF minimum probability used for v2 and v3 genetic ancestry group classification

In [7]:
v3_min_prob = 0.75
v2_min_prob = 0.9

### Load ONNX models

In [8]:
with hl.hadoop_open(gnomad_v2_onnx_rf, "rb") as f:
    v2_onx_fit = onnx.load(f)

with hl.hadoop_open(gnomad_v3_onnx_rf, "rb") as f:
    v3_onx_fit = onnx.load(f)

Initializing Hail with default parameters...
/opt/conda/miniconda3/lib/python3.10/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning: Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.
  warnings.warn(
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SPARKMONITOR_LISTENER: Started SparkListener for Jupyter Notebook
SPARKMONITOR_LISTENER: Port obtained from environment: 48881
SPARKMONITOR_LISTENER: Application Started: application_1689694192778_0002 ...Start Time: 1689696515125


Running on Apache Spark version 3.3.0
SparkUI available at http://jg1-m.c.broad-mpg-gnomad.internal:40497
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.119-ca0ff87b1687
LOGGING: writing to /home/hail/hail-20230718-1608-0.2.119-ca0ff87b1687.log


### Load gnomAD v3.1 loadings Hail Table and the VariantDataset to apply projection and genetic ancestry group assignment to

In [9]:
vds = hl.vds.read_vds(vds_to_project)
v3_loading_ht = hl.read_table(gnomad_v3_loadings)

2023-07-18 16:09:20.033 Hail: WARN: You are reading a VDS written with an older version of Hail.
  Hail now supports much faster interval filters on VDS, but you'll need to run either
  `hl.vds.truncate_reference_blocks(vds, ...)` and write a copy (see docs) or patch the
  existing VDS in place with `hl.vds.store_ref_block_max_length(vds_path)`.


### Load gnomAD v2.1 precomputed v2 PCA scores

As mentioned above, the v2 example will use our precomputed v2 PCA scores.

In [10]:
v2_meta_ht = get_gnomad_meta("exomes", full_meta=True)
v2_pcs_ht = v2_meta_ht.select(
    scores=hl.array([v2_meta_ht[f"PC{pc+1}"] for pc in range(v2_num_pcs)])
).select_globals()
v2_pcs_ht = v2_pcs_ht.filter(hl.is_defined(v2_pcs_ht.scores[0]))

## Perform PC projection using the v3.1 PCA loadings

### Create dense MatrixTable of only the variants found in the loadings Table

In [11]:
# Reduce variant data to only annotations needed for split and densify.
# This includes annotations needed for our standard genotype filter ('adj').
vds = hl.vds.VariantDataset(
    vds.reference_data, 
    vds.variant_data.select_entries("LA", "LGT", "GQ", "DP", "LAD")
)

# Split multiallelics.
vds = hl.vds.split_multi(vds, filter_changed_loci=True)

# Filter to variants in the loadings Table.
vds = hl.vds.filter_variants(vds, v3_loading_ht)

# Densify VDS.
mt = hl.vds.to_dense_mt(vds)

### We recommend filtering to entries meeting GQ, DP and het AB 'adj' thresholds

In [12]:
mt = filter_to_adj(mt)

### Checkpoint dense MT for PC projection

In [13]:
mt = mt.checkpoint(
    test_mt_output_path, 
    overwrite=not read_if_exists, 
    _read_if_exists=read_if_exists
)

### Project test dataset genotypes onto gnomAD v3.1 loadings and checkpoint the scores

In [14]:
# Project new genotypes onto loadings.
v3_pcs_ht = hl.experimental.pc_project(
    mt.GT, v3_loading_ht.loadings, v3_loading_ht.pca_af,
)

# Checkpoint PC projection results.
v3_pcs_ht = v3_pcs_ht.checkpoint(
    test_scores_output_path, 
    overwrite=not read_if_exists, 
    _read_if_exists=read_if_exists
)

2023-07-18 16:09:26.455 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


## Assign genetic ancestry group using ONNX RF model

### v3.1 RF model

In [15]:
ht, model = assign_population_pcs(
    v3_pcs_ht,
    pc_cols=v3_pcs_ht.scores[:v3_num_pcs],
    fit=v3_onx_fit,
    min_prob=v3_min_prob,
    apply_model_func = apply_onnx_classification_model,
)
ht = ht.checkpoint(
    gnomad_v3_assignment_path, 
    overwrite=not read_if_exists, 
    _read_if_exists=read_if_exists
)

ht.aggregate(hl.agg.counter(ht.pop))

INFO (gnomad.sample_qc.ancestry 369): Found the following sample count after population assignment: nfe: 378, oth: 32, afr: 28, amr: 60, eas: 42, sas: 49, asj: 25, fin: 35


{'afr': 28,
 'amr': 60,
 'asj': 25,
 'eas': 42,
 'fin': 35,
 'nfe': 378,
 'oth': 32,
 'sas': 49}

### v2.1 RF model

In [16]:
ht, model = assign_population_pcs(
    v2_pcs_ht,
    pc_cols=v2_pcs_ht.scores,
    fit=v2_onx_fit,
    min_prob=v2_min_prob,
    apply_model_func=apply_onnx_classification_model,
)
ht = ht.checkpoint(
    gnomad_v2_assignment_path, 
    overwrite=not read_if_exists, 
    _read_if_exists=read_if_exists
)

ht.aggregate(hl.agg.counter(ht.pop))

INFO (gnomad.sample_qc.ancestry 369): Found the following sample count after population assignment: fin: 14181, nfe: 74477, oth: 4631, amr: 21237, eas: 11842, sas: 17305, afr: 10425, asj: 5968


{'afr': 10425,
 'amr': 21237,
 'asj': 5968,
 'eas': 11842,
 'fin': 14181,
 'nfe': 74477,
 'oth': 4631,
 'sas': 17305}